In [36]:
import pandas as pd
import numpy as np
import glob

In [37]:
df = pd.read_csv('hlth_sha11_hc.csv')
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.head()

,freq,unit,icha11_hc,geo,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,A,EUR_HAB,TOT_HC,AT,3855.27,3928.93,4041.59,4129.77,4237.37,4359.69,4510.08,4690.47,4833.86,5498.80,5518.49,:
1,A,EUR_HAB,TOT_HC,BA,:,337.05,346.08,372.37,402.60,409.09,434.88,:,:,:,:,:
2,A,EUR_HAB,TOT_HC,BE,3651.59,3722.91,3814.10,3990.56,4096.58,4225.93,4372.70,4482.91,4498.15,4843.02,5104.86,:
3,A,EUR_HAB,TOT_HC,BG,436.06,413.42,457.59,471.74,510.22,556.20,586.55,625.59,753.65,890.50,989.78,:
4,A,EUR_HAB,TOT_HC,CH,6910.07,6977.19,7160.61,8311.17,8338.43,8315.14,8079.96,8586.70,9043.05,9482.43,10374.05,:


In [38]:
path = '../../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)
dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t', on_bad_lines='skip') for file in files}

In [39]:
for col in ["freq", "unit", "icha11_hc", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)
df.head()

,freq,unit,icha11_hc,country,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Annual,Euro per inhabitant,Current health care expenditure (CHE),Austria,3855.27,3928.93,4041.59,4129.77,4237.37,4359.69,4510.08,4690.47,4833.86,5498.80,5518.49,:
1,Annual,Euro per inhabitant,Current health care expenditure (CHE),Bosnia and Herzegovina,:,337.05,346.08,372.37,402.60,409.09,434.88,:,:,:,:,:
2,Annual,Euro per inhabitant,Current health care expenditure (CHE),Belgium,3651.59,3722.91,3814.10,3990.56,4096.58,4225.93,4372.70,4482.91,4498.15,4843.02,5104.86,:
3,Annual,Euro per inhabitant,Current health care expenditure (CHE),Bulgaria,436.06,413.42,457.59,471.74,510.22,556.20,586.55,625.59,753.65,890.50,989.78,:
4,Annual,Euro per inhabitant,Current health care expenditure (CHE),Switzerland,6910.07,6977.19,7160.61,8311.17,8338.43,8315.14,8079.96,8586.70,9043.05,9482.43,10374.05,:


In [40]:
df.replace(": z", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

In [41]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [42]:
df.head()

,freq,unit,icha11_hc,country,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Annual,Euro per inhabitant,Current health care expenditure (CHE),Austria,3855.0,3928.0,4041.0,4129.0,4237.0,4359.0,4510.0,4690.0,4833.0,5498.0,5518.0,NaN
1,Annual,Euro per inhabitant,Current health care expenditure (CHE),Bosnia and Herzegovina,NaN,337.0,346.0,372.0,402.0,409.0,434.0,NaN,NaN,NaN,NaN,NaN
2,Annual,Euro per inhabitant,Current health care expenditure (CHE),Belgium,3651.0,3722.0,3814.0,3990.0,4096.0,4225.0,4372.0,4482.0,4498.0,4843.0,5104.0,NaN
3,Annual,Euro per inhabitant,Current health care expenditure (CHE),Bulgaria,436.0,413.0,457.0,471.0,510.0,556.0,586.0,625.0,753.0,890.0,989.0,NaN
4,Annual,Euro per inhabitant,Current health care expenditure (CHE),Switzerland,6910.0,6977.0,7160.0,8311.0,8338.0,8315.0,8079.0,8586.0,9043.0,9482.0,10374.0,NaN


In [43]:
for col in df.columns:
    if not col.isdigit() and df[col].nunique() == 1:
        df.drop(columns=[col], inplace=True)

In [44]:
df.head()

,unit,country,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Euro per inhabitant,Austria,3855.0,3928.0,4041.0,4129.0,4237.0,4359.0,4510.0,4690.0,4833.0,5498.0,5518.0,NaN
1,Euro per inhabitant,Bosnia and Herzegovina,NaN,337.0,346.0,372.0,402.0,409.0,434.0,NaN,NaN,NaN,NaN,NaN
2,Euro per inhabitant,Belgium,3651.0,3722.0,3814.0,3990.0,4096.0,4225.0,4372.0,4482.0,4498.0,4843.0,5104.0,NaN
3,Euro per inhabitant,Bulgaria,436.0,413.0,457.0,471.0,510.0,556.0,586.0,625.0,753.0,890.0,989.0,NaN
4,Euro per inhabitant,Switzerland,6910.0,6977.0,7160.0,8311.0,8338.0,8315.0,8079.0,8586.0,9043.0,9482.0,10374.0,NaN


In [45]:
df.to_csv("./clean/hlth_sha11_hc.csv", index=False)